In [1]:
import os
import time
import pygame
import numpy as np
import streamlit as st
from gtts import gTTS
from datetime import datetime
from scipy.sparse import hstack
from googletrans import Translator
from sentence_transformers import SentenceTransformer
import joblib

pygame 2.6.1 (SDL 2.28.4, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\init\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---------- Setup ----------
# Initialize pygame mixer
pygame.mixer.init()

# Setup audio output folder
output_dir = "D:/tts_audio_playback"
os.makedirs(output_dir, exist_ok=True)

# Load ML components for genre prediction
clf = joblib.load('genre_classifier_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
mlb = joblib.load('mlb_transformer.pkl')
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Language options
languages = {
    "English": "en",
    "Urdu": "ur",
    "Arabic": "ar",
    "Korean": "ko"
}
translator = Translator()

# ---------- Streamlit UI ----------
st.title("🎬 Filmception: Movie Summary Translator & Genre Predictor")

summary = st.text_area("📜 Enter your movie summary:")
selected_options = st.multiselect("Choose what you want to do:", 
                                  ["Convert Summary to Audio", "Predict Genre"])

if summary and selected_options:
    if len(summary.split()) < 10:
        st.warning("✍️ Try entering a longer, more detailed summary for better results.")

    if "Convert Summary to Audio" in selected_options:
        lang_choice = st.selectbox("Select a language for audio playback:", list(languages.keys()))
        lang_code = languages[lang_choice]

        try:
            # Translate if needed
            if lang_code != 'en':
                st.write(f"🔁 Translating to {lang_choice}...")
                translated_text = translator.translate(summary, dest=lang_code).text
            else:
                translated_text = summary

            # Generate audio filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            audio_file = os.path.join(output_dir, f"summary_{timestamp}_{lang_code}.mp3")

            # Generate and save audio
            tts = gTTS(text=translated_text, lang=lang_code)
            tts.save(audio_file)

            st.success(f"✅ Audio generated in {lang_choice}!")

            # Download button
            with open(audio_file, "rb") as f:
                st.download_button("⬇️ Download Audio", f, file_name=os.path.basename(audio_file))

            # Play audio
            pygame.mixer.music.load(audio_file)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                time.sleep(0.5)
            pygame.mixer.music.stop()
        except Exception as e:
            st.error(f"⚠️ Error during TTS playback: {e}")

    if "Predict Genre" in selected_options:
        try:
            st.write("🔍 Predicting genres...")

            # Preprocess for ML model
            summary_tfidf = tfidf_vectorizer.transform([summary])
            summary_sbert = sbert_model.encode([summary])
            summary_combined = hstack([summary_tfidf, np.array(summary_sbert)])

            # Predict with threshold 0.4
            probas = clf.predict_proba(summary_combined)
            threshold = 0.4
            prediction = (probas >= threshold).astype(int)
            predicted_genres = mlb.inverse_transform(prediction)

            if predicted_genres[0]:
                st.success(f"🎯 Predicted Genres: {', '.join(predicted_genres[0])}")
            else:
                st.warning("🤔 No genre could be confidently predicted.")
        except Exception as e:
            st.error(f"⚠️ Error during genre prediction: {e}")
elif not summary:
    st.warning("⛔ Please enter a movie summary to continue.")

# Footer
st.markdown("---")
st.markdown("**Built with ❤️ using Streamlit, gTTS, Google Translate API, SBERT, and Scikit-learn.**")

2025-05-07 07:40:26.107 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.638 
  command:

    streamlit run C:\Users\init\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-07 07:40:26.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.642 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.642 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 07:40:26.643 Sessi

DeltaGenerator()